<a href="https://colab.research.google.com/github/MehrabHamidi99/ReverseEngineeringNetworks/blob/master/ReverseEngineeringDLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn

from sklearn.linear_model import LinearRegression, RANSACRegressor


In [ ]:
from numpy.core.memmap import dtype
class MLP_ReLU(nn.Module):

  Bias_std = 1

  def __init__(self, n_in, layer_list):
    super(MLP_ReLU, self).__init__()

    self.input_dim = n_in

    self.first_layer = nn.Sequential(nn.Linear(n_in, layer_list[0]), nn.ReLU())

    self.hidden_layers = nn.Sequential()
    for i in range(1, len(layer_list)):
      self.hidden_layers.append(nn.Linear(layer_list[i - 1], layer_list[i]))
      self.hidden_layers.append(nn.ReLU())

    self.apply(self.initialize_weights)

    self.output_dim = layer_list[-1]

    # self.last_layer = nn.Sequential(nn.Linear(layer_list[-1], n_out), nn.ReLU())
  
  def forward(self, x):
    first_layer_result = self.first_layer(x)
    output = self.hidden_layers(first_layer_result)
    return output

  def initialize_weights(self, layer):
    # Using He-normal and standard normal to initialize weights and biases
    if 'linear' in str(layer.__class__).lower():
      nn.init.kaiming_normal_(layer.weight)
      layer.weight = nn.Parameter(torch.tensor(layer.weight, dtype=torch.float64))
      nn.init.normal_(layer.bias)
      layer.bias = nn.Parameter(torch.tensor(layer.bias, dtype=torch.float64))

  def get_all_parameters(self):

    weights = []
    biases = []
    for k, v in self.state_dict().items():
      if 'weight' in k:
        weights.append(v.T)
      if 'bias' in k:
        biases.append(v)

    return weights, biases

In [ ]:
simple_network = MLP_ReLU(10, [20, 10, 10])

<ipython-input-2-5fb3359c4dec>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  layer.weight = nn.Parameter(torch.tensor(layer.weight, dtype=torch.float64))
<ipython-input-2-5fb3359c4dec>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  layer.bias = nn.Parameter(torch.tensor(layer.bias, dtype=torch.float64))


In [ ]:
simple_network.output_dim

10

In [ ]:
class LinearRegion1D:

    def __init__(self, param_min, param_max, fn_weight, fn_bias, next_layer_off):

        # Region a^Tx + b = 0
        '''
        a: fn_weight
        b: bn_bias
        '''
        self._min = param_min
        self._max = param_max
        self._fn_weight = fn_weight
        self._fn_bias = fn_bias
        self._next_layer_off = next_layer_off

    def get_new_regions(self, new_weight_n, new_bias_n, n):
        weight_n = torch.dot(self._fn_weight, new_weight_n) # <a, a'>
        bias_n = torch.dot(self._fn_bias, new_weight_n) + new_bias_n # <b, a'> + b

        if weight_n == 0:
            min_image = bias_n
            max_image = bias_n
        elif weight_n >= 0:
            min_image = weight_n * self._min + bias_n
            max_image = weight_n * self._max + bias_n
        else:
            min_image = weight_n * self._max + bias_n
            max_image = weight_n * self._min + bias_n
        if 0 < min_image:
            return [self]
        elif 0 > max_image:
            self._next_layer_off.append(n)
            return [self]
        else:
            if weight_n == 0:
                return [self]
            else:
                preimage = (-bias_n) / weight_n
                next_layer_off0 = list(np.copy(self._next_layer_off))
                next_layer_off1 = list(np.copy(self._next_layer_off))
                if weight_n >= 0:
                    next_layer_off0.append(n)
                else:
                    next_layer_off1.append(n)
                region0 = LinearRegion1D(self._min, preimage, self._fn_weight, self._fn_bias, next_layer_off0)
                region1 = LinearRegion1D(preimage, self._max, self._fn_weight, self._fn_bias, next_layer_off1)
                return [region0, region1]

    def next_layer(self, new_weight, new_bias):
        # self._fn_weight = np.dot(self._fn_weight, new_weight.T).ravel()
        self._fn_weight = self._fn_weight @ new_weight
        self._fn_bias = self._fn_bias @ new_weight + new_bias
        self._fn_weight[self._next_layer_off] = 0
        self._fn_bias[self._next_layer_off] = 0
        self._next_layer_off = []

    @property
    def max(self):
        return self._max

    @property
    def min(self):
        return self._min

    @property
    def fn_weight(self):
        return self._fn_weight

    @property
    def fn_bias(self):
        return self._fn_bias

    @property
    def next_layer_off(self):
        return self._next_layer_off

    @property
    def dead(self):
        return np.all(np.equal(self._fn_weight, 0))


In [ ]:
class MultipleLinearRegression():
    ''' Class that implements Multiple Linear Regression '''
    def __init__(self):
        self.b = 0
        self.w = []
        
    def fit(self, X, y):
        # If there is only one feature we need to reshape input.
        if len(X.shape) == 1:
            X.reshape(-1, 1)
            
        # Add 'ones' to model coefficient b in data.
        ones = np.ones(shape=X.shape[0]).reshape(-1, 1)
        X = np.concatenate((ones, X), 1)

        print(X.shape)


        print(X.transpose().dot(X).shape)
        print(np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).shape)
        print("----")
        coeficients = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)
        self.b = coeficients[0]
        self.w = coeficients[1:]

    def predict(self, X):
        predictions = []
        for x in X:
            prediction = self.b

            for xi, wi in zip(x, self.w):
                prediction += wi * xi
            
            predictions.append(prediction)
            
        return predictions

In [ ]:
INPUT_DIM = 10

SAMPLE_RADIUS = int(10 * np.sqrt(INPUT_DIM)) # 10 * √d
SAMPLE_LENGTH = int(500 * np.sqrt(INPUT_DIM)) # 500 * √d

PRECISION_LINE = 0.00001
PRECISION_BOUNDARY = 0.0001

ITERATIONS = 15
EPS = 0.001
CHECK_EPS = 0.001
DEDUPLICATION_EPS = 0.001

torch.random.seed = 32
np.random.seed = 32

APPROX_RADIUS = 0.2
APPROX_NUM = int(1.5 * INPUT_DIM)
APPROX_THRESHOLD = 0.9

MULTIPLE_POINTS = False  # Option for tuning approx_boundary, default is False.

In [ ]:
def approximate_1d(model, end_point1, end_point2, iterations_, eps, init_samples=1, precision=1e-5, use_outputs=None, single_search=False):
  """
  This function sample some points in the [0, 1] intnerval and query the netowkr and calculate the slopes of the sorted queries and 
  keep the slopes which are distinct.
  Then do the binary search over the sampled points to find
  the correct hyperplane boundaaries basesd on the queries (different slopes results from the model)

  There's two option single point and multiple point search, in multiple point we query the network a lot!

  INPUTS:
  model -> a pytorch model
  end_point1, end_point2 -> (1d torch tensor) line segment query
  iterations_ -> (int) number of iterations for the search queries
  eps -> (float) epsilon threshold for comaring the slopes
  init_samples -> (int) number of the samples to begin with
  precision -> (float) epsilon threshold for considering two query point distinct or not

  RETURN:
  (points belong to different hyperplane on the line segment, number of sampled pointss)
  """

  with torch.no_grad():

    end_point1 = end_point1.reshape(1, -1) # Making 2d array
    end_point2 = end_point2.reshape(1, -1)
    samples_t = torch.tensor(np.arange(0, 1.0000000001, 1./(init_samples + 1), dtype=np.float64), dtype=torch.float64) # sample points in [0, 1] with same division of the intnerval 
    total_samples = 0

    for iter in range(iterations_):
      num_samples = len(samples_t)
      samples = torch.repeat_interleave(end_point1, repeats=num_samples, dim=0) + (samples_t.reshape(-1, 1) * (end_point2 - end_point1)) # ep1 + t(ep1 - ep2) \forall t
      outcomes = model(samples) # get the query of the model

      total_samples += num_samples
      output_dim = outcomes.shape[1]

      if use_outputs == None:
          outputs_used = output_dim
      else:
          outputs_used = use_outputs

      # Calculating the slopes for all the output dim  of the hyperplanes: (f(x) - f(y)) / (x - y) for queries      
      slopes = torch.abs(torch.divide(outcomes[1:,] - outcomes[:-1,],  torch.repeat_interleave((samples_t[1:] - samples_t[:-1])[:, None], repeats=model.output_dim, dim=1)))[:,:-1].T
      # Check epsilon difference between calculated slopes
      diff = torch.logical_or(torch.less(slopes[:, 1:], (1 - eps) * slopes[:, :-1]), torch.greater(slopes[:, 1:], (1 + eps) * slopes[:, :-1]))
      diff = torch.any(diff, dim=0)
      if single_search:
        diff_indices = torch.nonzero(diff)[0] # signle search
      else:
        diff_indices = torch.nonzero(diff) # multiple search
      end_t = samples_t[[0, -1]]
      keep_t = samples_t[diff_indices + 1]

      # Sampling middle points
      if iter < iterations_ - 1:
        new_t_1 = (2 * keep_t + samples_t[diff_indices]) / 3
        new_t_2 = (2 * keep_t + samples_t[diff_indices + 2]) / 3
        if single_search:
          samples_t, _ = torch.sort(torch.hstack((end_t, keep_t, new_t_1, new_t_2))) # single search
        else:
          samples_t, _ = torch.sort((torch.vstack((end_t[:, None], keep_t, new_t_1, new_t_2))).flatten()) # multiple search

      # print(samples_t)


    # Compare the difference between sample vectors to the uniqueness threshold
    if len(samples_t) > 0:
      if single_search:
        unique = np.nonzero(samples_t[1:] - samples_t[:-1] > precision)[0] # single search
      else:
        unique = torch.nonzero(samples_t[1:] - samples_t[:-1] > precision).flatten() # multiple search
      output = 0.5 * (samples_t[torch.hstack((unique, torch.tensor([-1])))] + samples_t[torch.hstack((torch.tensor([0]), unique + 1))])
    else:
        output = samples_t

    return output, total_samples

def regions_1d(the_weights, the_biases, endpt1, endpt2):
    '''
    This function cosntruct all of the linear hyperplane of a network with its given weights and biases, over a line segment which is specified with
    the endpt1, endpt2

    INPUTS:
    the_weights -> (torch.tensor(2d))
    the biases -> (torch.tensor(3d))
    endpt1, endpt2 -> 2d tensors, line segment


    RETURN:
    [Regions]
    '''
    regions = [LinearRegion1D(param_min=0., param_max=1., fn_weight=(endpt2 - endpt1), fn_bias=endpt1,
                              next_layer_off=[])]
    depth = len(the_weights)

    for k in range(depth - 1):
        for n in range(the_biases[k].shape[0]):
            new_regions = []
            for region in regions:
                new_regions = new_regions + region.get_new_regions(the_weights[k][:, n], the_biases[k][n], n)
            regions = new_regions
        for region in regions:
            region.next_layer(the_weights[k], the_biases[k])
    for region in regions:
        region.next_layer(the_weights[-1], the_biases[-1])
    return regions


def region_pts_1d(regions, param_min=-torch.inf):
    '''
    This function return the points alogside the line segment for all the linear depth 1 linear regions of the network

    INPUTS:
    regions -> [Regions]

    RETURN:
    ([int], [int]), the x's length of the query line segment (endpoint1 - endpoint2) * x + endpoint1
    '''
    xs = []
    ys = []
    for region in regions:
        if region.min == param_min:
            pass
        else:
            xs.append(region.min)
            ys.append(region.min * region.fn_weight + region.fn_bias)
    return xs, ys


def calculate_exact_1d(model, end_point1, end_point2):
  '''
  This function calculate adn return exact boundary points of first depth alongside of the linen sesgmennt

  INPUTS:
  model -> pytorch model
  endpoint1, endpoint2 -> torch.tensor, line segmeent

  RETURN:
  ([int]), exact boundary points of the netowkrs
  '''
  with torch.no_grad():
    the_weights, the_biases = model.get_all_parameters()
    # the_biases = model.first_layer.state_dict()['0.bias']

    # Construct all the linear regions, given weights and biases alongside a given line segment
    exact_regions = regions_1d(the_weights, the_biases, end_point1, end_point2)

    points, _ = region_pts_1d(exact_regions, 0)

    return points

def approx_boundary(model, point, radius, num_samples, threshold, iterations, eps, on_pos_side_of=None, init_samples=1, precision=1e-4, use_outputs=None, multiple_points = False):
  dim = model.input_dim
  results = []
  i = 0
  total_samples = 0

  while len(results) < num_samples:
    if len(results) == 0 and i > 10 * num_samples:  # Heuristic stopping point
        print("Exceeded maximum number of samples")
        return None, None, None, None, total_samples
    i += 1

    midpoint = torch.rand((dim,)) - 0.5 # Sample a point from interval [-0.5, 0.5); M
    midpoint /= torch.linalg.norm(midpoint) # Make it a unit vector

    perp = torch.rand((dim,)) - 0.5 # v
    perp = perp - torch.dot(midpoint, perp) * midpoint # v - (v^T . M) * M      Why elementwise multiplication?!
    perp /= torch.linalg.norm(perp)

    endpt1 = point + radius * midpoint + int(50 * np.sqrt(dim)) * perp
    endpt2 = point + radius * midpoint - int(50 * np.sqrt(dim)) * perp
    
    if type(on_pos_side_of) is not tuple:
      output, samples = approximate_1d(model, endpt1, endpt2, iterations, eps, init_samples=init_samples,
                                    precision=precision, use_outputs=use_outputs, single_search=False)
      total_samples += samples

      if len(output[1:-1]) > 0:
        output = output[1:-1]
        if len(results) == 0:
          results = torch.repeat_interleave(output[None, :], repeats=endpt2.shape[0], dim=0).T * torch.repeat_interleave((endpt2 - endpt1)[None, :], repeats=output.shape[0], dim=0) # appennding result length to investigate line inn between a * (x_1 - x_2) + x_1
        else:
          results = torch.cat([results, torch.repeat_interleave(output[None, :], repeats=endpt2.shape[0], dim=0).T * torch.repeat_interleave((endpt2 - endpt1)[None, :], repeats=output.shape[0], dim=0)], dim=0)
  
  X = results[:, :-1]
  y = results[:, -1]

  mlr = RANSACRegressor(random_state=0).fit(X, y)
  weight = np.hstack((mlr.estimator_.coef_, -1))
  bias = mlr.estimator_.intercept_

  bias /= np.linalg.norm(weight)
  weight /= np.linalg.norm(weight)

  return weight, bias, results[0, :], results, total_samples


In [ ]:

########

approx_num1 = 0
approx_weight1 = []
approx_bias1 = []
unused = []

def algorithm(model, approx_num1, approx_weight1, approx_bias1, unused):
  layer1_samples = 0
  # Sample  midpoint at a certain radius

  midpoint = torch.rand((INPUT_DIM,)) - 0.5 # Sample a point from interval [-0.5, 0.5); M
  midpoint /= torch.linalg.norm(midpoint) # Make it a unit vector

  perp = torch.rand((INPUT_DIM,)) - 0.5 # v
  perp = perp - torch.dot(midpoint, perp) * midpoint # v - (v^T . M) * M      Why elementwise multiplication?!
  perp /= torch.linalg.norm(perp)

  endpt1 = SAMPLE_RADIUS * midpoint + (SAMPLE_LENGTH / 2) * perp # rM + lv
  endpt2 = SAMPLE_RADIUS * midpoint - (SAMPLE_LENGTH / 2) * perp # rM - lv

  # Approximate region transition points along sample segment, and check against exact answer.
  approx_pts, samples = approximate_1d(model, endpt1, endpt2, ITERATIONS, EPS)

  layer1_samples += samples

  exact_pts = calculate_exact_1d(model, torch.tensor(endpt1, dtype=torch.float64), torch.tensor(endpt2, dtype=torch.float64))

  # Check the precision of the estimation  
  # for ex_pt in exact_pts:
  #   if torch.all(torch.abs(approx_pts - ex_pt) > 1e-5):
  #     print(ex_pt)
  #     print("ERORR")

  print("Estimated num pts along line: %d, true num pts: %d" % (len(approx_pts), len(exact_pts)))

  ##########

  for point_t in approx_pts:
    point = (endpt2 - endpt1) * point_t + endpt1

    # Check if this point lies on a hyperplane already found.
    repeated = False
    for m in range(approx_num1):
        if not repeated:
            value = np.dot(approx_weight1[m], point) + approx_bias1[m]
            value_threshold = CHECK_EPS * np.linalg.norm(point)
            if value_threshold > value > -value_threshold:
                repeated = True

    if not repeated: # and not (approx_num1 == network[0] and KNOWN_ARCHITECTURE):
      # Find the boundary in the neighborhood of each point by sampling more transition points.
      # and fitting a hyperplane.
      weight, bias, _, pts, samples = approx_boundary(model, point, APPROX_RADIUS, APPROX_NUM,
                                                      APPROX_THRESHOLD, ITERATIONS, EPS,
                                                      precision=PRECISION_BOUNDARY,
                                                      multiple_points=MULTIPLE_POINTS)
      layer1_samples += samples
      raise Exception()
      if np.array(weight).shape:  # Check that a valid hyperplane was found.
          valid = True
          i = 0
          extra_pts = []

          # Samples points a certain distance away to see if the hyperplane extends that far unchanged.
          while valid and i < CHECK_NUM:
              check_point = np.random.random((INPUT_DIM,)) - 0.5
              check_point = check_point - weight * np.dot(weight, check_point)
              check_point = point + CHECK_RADIUS * (check_point / np.linalg.norm(check_point))
              check_vec = CHECK_RADIUS * CHECK_EPS * weight
              straight = is_straight(sess, check_point, check_vec, EPS)
              layer1_samples += 3
              if straight:
                  valid = False
                  unused.append((point, weight, bias))
              i += 1

          # If passes checks, add candidate hyperplane to list for layer 1.
          if valid:
              # Check through proposed neurons for layer 1 for duplicate weights.
              duplicate = False
              for n in range(0, approx_num1):
                  if np.linalg.norm(np.abs(approx_weight1[n]) - np.abs(weight)) < DEDUPLICATION_EPS:
                      duplicate = True
                      break
              if not duplicate:
                  approx_weight1.append(weight)
                  approx_bias1.append(bias)
                  approx_num1 += 1
        # elif not repeated:
          unused.append((point, None, None))


In [ ]:
algorithm(simple_network, approx_num1, approx_weight1, approx_bias1, unused)

Estimated num pts along line: 42, true num pts: 30


<ipython-input-69-89f14224f0d1>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  exact_pts = calculate_exact_1d(model, torch.tensor(endpt1, dtype=torch.float64), torch.tensor(endpt2, dtype=torch.float64))


Exception: ignored